<a href="https://colab.research.google.com/github/mohaljamal/hu-bby261-2020/blob/main/proje_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from IPython.display import HTML, Image
from google.colab.output import eval_js
from base64 import b64decode
from PIL import Image
import io

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD

from keras.datasets import mnist
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.models import load_model




def LoadData():	
	(trainX, trainY), (testX, testY) = mnist.load_data()	
	trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
	testX = testX.reshape((testX.shape[0], 28, 28, 1))
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY
 

def NormalizePixels(train, test):	
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')	
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0	
	return train_norm, test_norm
 

def DefineModel():
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
	model.add(Dense(10, activation='softmax'))
	opt = SGD(lr=0.01, momentum=0.9)
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model
 

def CreateAndSaveModel():
	trainX, trainY, testX, testY = LoadData()
	trainX, testX = NormalizePixels(trainX, testX)	
	model = DefineModel()
 
  #takes long time about 10 mins
	model.fit(trainX, trainY, epochs=4, batch_size=32, verbose=0)	
	model.save('ClassificationModel.h5')

def TestModel():
	trainX, trainY, testX, testY = LoadData()
	trainX, testX = NormalizePixels(trainX, testX)
	model = load_model('ClassificationModel.h5')
	_, accuracy= model.evaluate(testX, testY, verbose=0)
	print('> Accuracy =  %.3f ' % (accuracy * 100.0))


canvas_html = """
<canvas width=256 height=256 style="border:2px solid #cdcdcd;  "></canvas>
</br>
<button>Evaluate</button>
<button id="clear">Clear</button>
<script>
var canvas = document.querySelector('canvas')
var ctx = canvas.getContext('2d')
ctx.lineWidth = 15
ctx.fillStyle = "black";
ctx.fillRect(0, 0, canvas.width, canvas.height);
ctx.strokeStyle = "#fff"
var button = document.querySelector('button')
var mouse = {x: 0, y: 0}
canvas.addEventListener('mousemove', function(e) {
  mouse.x = e.pageX - this.offsetLeft
  mouse.y = e.pageY - this.offsetTop
})
canvas.onmousedown = ()=>{
  ctx.beginPath()
  ctx.moveTo(mouse.x, mouse.y)
  canvas.addEventListener('mousemove', onPaint)
}
canvas.onmouseup = ()=>{
  canvas.removeEventListener('mousemove', onPaint)
}
var onPaint = ()=>{
  ctx.lineTo(mouse.x, mouse.y)
  ctx.stroke()
}
var data = new Promise(resolve=>{
  button.onclick = ()=>{
    resolve(canvas.toDataURL('image/png'))
  }
})

document.getElementById("clear").onclick = ()=> {
  const context = canvas.getContext('2d');

    context.clearRect(0, 0, canvas.width, canvas.height);
    context.fillStyle = "black";
    context.fillRect(0, 0, canvas.width, canvas.height);
    context.strokeStyle = "#fff"
}
</script>
"""

def Draw(filename='big.png'):
  display(HTML(canvas_html))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  image = Image.open(io.BytesIO(binary))
  img28 = image.resize((28, 28), Image.ANTIALIAS)
  img28.save("sample_image.png")
  with open(filename, 'wb') as f:
    f.write(binary)
  prediction = Predict()
  print( "Prediction =  %d" % prediction)

  
  


def LoadAndConvertImage(filename):

	img = load_img(filename, color_mode = "grayscale", target_size=(28, 28))
	img = img_to_array(img)
	img = img.reshape(1, 28, 28, 1)
	img = img.astype('float32')
	img = img / 255.0
	return img
 

def Predict():
  img = LoadAndConvertImage('sample_image.png')
  model = load_model('ClassificationModel.h5')
  digit = model.predict_classes(img)

  return digit[0]
 


#bu fonksiyon bir defa calistirmasi gerikyor sonra gerek yok cunku model dosya olarak kaydiyor
CreateAndSaveModel() 
TestModel()
Draw()

> Accuracy =  98.660 


Prediction =  5
